In [ ]:
import gc
import os
import urllib.request
from pathlib import Path

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
from scipy import signal

In [ ]:
# plots
plt.style.use("seaborn-talk")

In [ ]:
project = Path.cwd()
output = project / "output"
input = project / "input"
output_data = output / "data"
figures = project / "figures"

In [ ]:
# get MS River input discharge
conversion = 1 / (1000 ** 3)  # m^3 -> km^3
t0 = pd.to_datetime("2018-01-01 01:00:00")
tf = pd.to_datetime("2018-11-23 00:00:00")
belle_chasse_input = pd.read_csv(
    output / "delft3d_input/mississippi_at_belle_chase_2017-12-01_godin.csv",
    sep="\t",
    index_col=[0],
    parse_dates=True
).loc[t0:tf]

# integrate piece-wise discharge observations assuming observation held constant for previous hour. Convert to km/year
belle_chasse_total = (belle_chasse_input * 3600).sum().values * conversion

In [ ]:
model = "p03"
model_output = Path(
    f"/mnt/e/MS_River_plume/Delft3D/models/prod_2018/{model}_10sig_2018prod/"
)
merged_his_fn = model_output / f"{model}_merged_his.nc"

variables = [
    "cross_section_discharge",
    "cross_section_geom",
    "cross_section_cumulative_discharge",
    "cross_section_velocity",
]

his = xr.open_dataset(merged_his_fn)[variables]
his = his.assign_coords(cross_section_name=his.cross_section_name.astype("unicode"))
his = his.swap_dims({"cross_section": "cross_section_name"})

In [ ]:
cross_sections = ["ms_buras", "allison_39", "allison_15-3", "allison_8-4", "allison_4-2"]
outlets = [
    "ostrica",
    "fort_st_philip",
    "baptiste_collette_1",
    "grand_pass",
    "west_bay_diversion",
    "cubits_gap",
    "south_west_pass",
    "south_pass",
    "pass_a_loutre",
]

In [ ]:
fig, ax = plt.subplots()
var = "cross_section_discharge"
time = his["time"].values
for cross_section in cross_sections:
    data = his.sel(cross_section_name=cross_section)[var].copy().values

    ax.plot(time, data, label=f"{cross_section}")
    pass

data = his.sel(cross_section_name="south_west_pass")[var].copy().values
for outlet in outlets[-2:]:
    _data = his.sel(cross_section_name=outlet)[var].copy().values
    data += _data

ax.plot(time, data, label=f"Head of Passes")

ax.set_xlim([pd.to_datetime("2018-01-02"), his.time[-1]])
ax.grid()
ax.set_ylabel("discharge [$m^3/s$]")
ax.legend(ncol=2)

In [ ]:
fig, ax = plt.subplots()
downstream = "allison_15-3"
sub_outlets = outlets[1:4]

upstream = his.sel(cross_section_name="allison_39")[var].copy().values
downstream = his.sel(cross_section_name="allison_15-3")[var].copy().values
fort_st_philip = his.sel(cross_section_name="fort_st_philip")[var].copy().values
baptiste_collette_1 = his.sel(cross_section_name="baptiste_collette_1")[var].copy().values
grand_pass = his.sel(cross_section_name="grand_pass")[var].copy().values

ax.plot(time, upstream, label=f"allison_39 (upstream)")
ax.plot(time, downstream, label=f"allison_15-3 (downstream)")
ax.plot(time, fort_st_philip, label=f"fort_st_philip (1st outlet)")
ax.plot(time, baptiste_collette_1, label=f"Baptiste Collette (2nd outlet)")
ax.plot(time, grand_pass, label=f"Grand Pass (3rd outlet)")
ax.plot(time, fort_st_philip + baptiste_collette_1 + grand_pass + downstream, label=f"outlet + downstream")

ax.legend(ncol=3)

In [ ]:
# annual water discarge [Allison et al. 2012] km/y
allison_belle_chasse = 527
allison_Q_data = {
    # "bohemia": 1,
    "ostrica": 8,
    "fort_st_philip": 31,
    "baptiste_collette_1": 49,
    "grand_pass": 52,
    "west_bay_diversion": 33,
    # "small_cuts": 10,
    "cubits_gap": 52,
    "south_west_pass": 163,
    "south_pass": 47,
    "pass_a_loutre": 43,
}

allison_Q = pd.DataFrame.from_dict(
    data=allison_Q_data, orient="index", columns=["Allison et al. (2012)"]
)
loss = allison_belle_chasse - allison_Q.sum()
allison_Q.loc["unaccounted"] = loss

In [ ]:
model_annual_discharge = (
    (
        (his["cross_section_discharge"] * (10 * 60))
        .sum(dim="time")
        .sel(cross_section_name=outlets)
        * conversion
    )
    .to_dataframe()
    .rename(columns={"cross_section_discharge": "Model"})
)
loss = belle_chasse_total - model_annual_discharge.sum()
model_annual_discharge.loc["unaccounted"] = loss

In [ ]:
# make dataframe
annual_discharge = model_annual_discharge.join(allison_Q)
rel_annual_discharge = (model_annual_discharge / belle_chasse_total).join(
    allison_Q / allison_belle_chasse
)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
cross_sections = annual_discharge.index.values
(100 * rel_annual_discharge).plot.bar(ax=ax, zorder=10)

xaxis_labels = [
    "Ostrica",
    "Fort St. Philip",
    "Baptiste Collette",
    "Grand Pass",
    "West Bay Diversion",
    "Cubits Gap",
    "South West Pass",
    "South Pass",
    "Pass a Loutre",
    "Unaccounted For",
]


ax.set_xlabel("")
ax.set_ylabel("Percent Annual Discharge [%]")
ax.tick_params(axis="x", rotation=45)
ax.set_xticklabels(xaxis_labels)
ax.grid(zorder=0)
ax.set_ylim([0, 35])
ax.legend(loc="upper left")
ax.set_title("Annual discharge relative to Belle Chasse")
fig.savefig(
    figures / f"percent_annual_discharge_belle_chase_{model}", bbox_inches="tight"
)